In [1]:
#import packages
import pandas as pd # data manipulation
import numpy as np
from scipy import sparse

import re
import string
import time
import seaborn as sns
import itertools

# Load Data

In [2]:
#air_visit_data.csv: historical visit data for the air restaurants as main training dataset
train_data = pd.read_csv('../input/air_visit_data.csv')

#store info
air_store_info = pd.read_csv('../input/air_store_info.csv')
hpg_store_info = pd.read_csv('../input/hpg_store_info.csv')

#reservation info
air_reserve = pd.read_csv('../input/air_reserve.csv')
hpg_reserve = pd.read_csv('../input/hpg_reserve.csv')

store_id_relation = pd.read_csv('../input/store_id_relation.csv')
test_data = pd.read_csv('../input/sample_submission.csv')
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
train_size = train_data.shape[0]

# Data Preprocessing

### Display and merge train & test data for further feature engineering

In [3]:
display(train_data.head())
display(test_data.head())

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [4]:
#split store id and visit date in test_data id
test_data['visit_date'] = test_data['id'].map(lambda x: x.split('_')[2])
test_data['air_store_id'] = test_data['id'].map(lambda x: '_'.join(x.split('_'[:2])))
#test_data.drop(['id'], axis = 1, inplace = True)

In [5]:
test_data.head()

,id,visitors,visit_date,air_store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9_2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9_2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9_2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9_2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9_2017-04-27


In [6]:
#merge train_data and test_data
full_data = pd.concat([train_data, test_data]).reset_index(drop = True)

In [7]:
full_data.head()

,air_store_id,id,visit_date,visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25
1,air_ba937bf13d40fb24,NaN,2016-01-14,32
2,air_ba937bf13d40fb24,NaN,2016-01-15,29
3,air_ba937bf13d40fb24,NaN,2016-01-16,22
4,air_ba937bf13d40fb24,NaN,2016-01-18,6


### Exploration of Datetime Features
Get more information about visit time, eg. the day of week/ is month end

In [8]:
#change data type from string to datetime
full_data['visit_date'] = pd.to_datetime(full_data['visit_date'])

In [9]:
#preprocessing datetime data using pandas.Series.dt
full_data['dow'] = full_data['visit_date'].dt.dayofweek
full_data['year'] = full_data['visit_date'].dt.year
full_data['month'] = full_data['visit_date'].dt.month
full_data['doy'] = full_data['visit_date'].dt.dayofyear
full_data['dom'] = full_data['visit_date'].dt.days_in_month
full_data['woy'] = full_data['visit_date'].dt.weekofyear
full_data['is_month_end'] = full_data['visit_date'].dt.is_month_end
full_data['visit_date'] = full_data['visit_date'].dt.date

In [10]:
# change the format from 20160113 to "20160113"
full_data['date_int'] = full_data['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

### Exploration of Area Features
Split area names in store information (with space)
<p>eg. 'Hyōgo-ken Kōbe-shi Kumoidōri' contains three levels of area info: 1.Hyōgo-ken; 2.Kōbe-shi; 3.Kumoidōri

In [11]:
air_store_info['air_area_lv1'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[0])
air_store_info['air_area_lv2'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[1])
air_store_info['air_area_lv3'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[2])

hpg_store_info['hpg_area_lv1'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[0])
hpg_store_info['hpg_area_lv2'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[1])
hpg_store_info['hpg_area_lv3'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[2])

In [12]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen


Create features that based on different levels of area

In [13]:
#For air_store_info

# Groupby and calculate the number of stores in same area (with different level of areas)
# Named with 'air_stores_on_same_addr' & 'air_stores_lv1' &'air_stores_lv2'
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['latitude','longitude']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


In [24]:
#Calculate the mean/max/min of level-1 latitude
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

#Calculate the mean/max/min of level-1 longitude
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

In [26]:
#Calculate the mean/max/min of level-2 latitude and longitude
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

In [27]:
#Same for hpg_store_info

# Groupby and calculate the number of stores in same area (with different level of areas)
# Named with 'air_stores_on_same_addr' & 'air_stores_lv1' &'air_stores_lv2'
hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['latitude','longitude']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

#Calculate the mean/max/min of level-1 latitude and longitude
hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

#Calculate the mean/max/min of level-2 latitude and longitude
hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

### Merge store information for stores that exist in both air and hpg

In [28]:
air_store_info = pd.merge(air_store_info, store_id_relation, how='left', on='air_store_id')

In [29]:
air_store_info

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,...,mean_lat_air_lv1_y,max_lat_air_lv1,min_lat_air_lv1,mean_lat_air_lv2,max_lat_air_lv2,min_lat_air_lv2,mean_lon_air_lv2,max_lon_air_lv2,min_lon_air_lv2,hpg_store_id
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,34.741237,34.815149,34.688241,34.700598,34.720228,34.688241,135.213229,135.265455,135.187254,NaN
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,34.741237,34.815149,34.688241,34.700598,34.720228,34.688241,135.213229,135.265455,135.187254,hpg_9b38b9e13da6da27
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,34.741237,34.815149,34.688241,34.700598,34.720228,34.688241,135.213229,135.265455,135.187254,NaN
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,34.741237,34.815149,34.688241,34.700598,34.720228,34.688241,135.213229,135.265455,135.187254,NaN
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,35.675963,35.775664,35.546631,35.660476,35.674918,35.658068,139.748928,139.751599,139.729426,NaN
5,air_99c3eae84130c1cb,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,35.675963,35.775664,35.546631,35.660476,35.674918,35.658068,139.748928,139.751599,139.729426,NaN
6,air_f183a514cb8ff4fa,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,35.675963,35.775664,35.546631,35.660476,35.674918,35.658068,139.748928,139.751599,139.729426,NaN
7,air_6b9fa44a9cf504a1,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,35.675963,35.775664,35.546631,35.660476,35.674918,35.658068,139.748928,139.751599,139.729426,NaN
8,air_0919d54f0c9a24b8,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,35.675963,35.775664,35.546631,35.660476,35.674918,35.658068,139.748928,139.751599,139.729426,NaN
9,air_2c6c79d597e48096,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,35.675963,35.775664,35.546631,35.660476,35.674918,35.658068,139.748928,139.751599,139.729426,NaN


In [ ]:
😂😂😂🤣emoji test!